In [1]:
from collections import defaultdict
import numpy as np
import pandas as pd
import copy


In [2]:
bookPath = "goblet_book.txt"

with open(bookPath) as f:
    bookContent = f.read() #string

alphabet = sorted(set(bookContent))
K = len(alphabet)
K

83

In [3]:
index_to_char = {i: char for i, char in enumerate(alphabet)}
char_to_index = {char: i for i, char in enumerate(alphabet)}


In [4]:
def string2vec(string):
    vec = np.array([char_to_index[c] for c in string])
    return vec

def oneHotEncode(idx):
    N = len(idx) if type(idx) is np.ndarray else 1
    one_hot_matrix = np.zeros((N, K))
    one_hot_matrix[np.arange(N), idx] = 1
    return one_hot_matrix.T

def string2oneHot(string):
    vec = string2vec(string)
    return oneHotEncode(vec)

def oneHot2String(oneHot):
    vec = np.argmax(oneHot, axis=0)
    string = ''.join([index_to_char[idx] for idx in vec])
    return string

oneHot = string2oneHot("Hi")
print(oneHot.shape)
print(oneHot.T)

string = oneHot2String(oneHot)
print(string)


(83, 2)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Hi


In [5]:

global num_grads
#num_grads = Grads(1,1)

class Weights:
    def __init__(self, chars, hiddenDim, sig):
        self.b = np.zeros((hiddenDim, 1))
        self.c = np.zeros((chars, 1))
        self.U = np.random.random((hiddenDim, chars)) * sig
        self.W = np.random.random((hiddenDim, hiddenDim)) * sig
        self.V = np.random.random((chars, hiddenDim)) * sig

class Grads:
    def __init__(self, chars, hiddenDim):
        self.b = np.zeros((hiddenDim, 1))
        self.c = np.zeros((chars, 1))
        self.U = np.zeros((hiddenDim, chars))
        self.W = np.zeros((hiddenDim, hiddenDim))
        self.V = np.zeros((chars, hiddenDim))

class RNN:
    def __init__(self, chars, hiddenDim, seqLength, eta, sig = 0.01):
        self.chars = chars
        self.hiddenDim = hiddenDim
        self.seqLength = seqLength
        self.eta = eta
        self.sig = sig

        self.weights = Weights(chars, hiddenDim, sig)
        self.grads = Grads(chars, hiddenDim)

        self.h0 = np.zeros((hiddenDim, 1))
        self.dummy = "."

        self.cache = defaultdict(list)

    def relu(self, X, d=False):
        if not d:
            return np.maximum(0, X)
            # np.where(X > 0, X, 0)
        else:
            return np.where(X > 0, 1, 0)

    def tanh(self, X, d=False):
        if not d:
            return (np.exp(X) - np.exp(-X)) / (np.exp(X) + np.exp(-X))
        else:
            return 1 - self.tanh(X)**2


    def softmax(self, x: np.ndarray) -> np.ndarray:
        """ Standard definition of the softmax function """
        exps = np.exp(x)
        return exps / np.sum(exps, axis=0)

    def SingleForwardPass(self, X, h_prev):
        assert X.shape[0] == self.chars

        a = self.weights.W @ h_prev    # 100x100 @ 100x1 = 100x1
        a += self.weights.U @ X        # + 100x83 @ 83x1 = 100x1
        a += self.weights.b            # + 100x1
        h = self.tanh(a)

        o = self.weights.V @ h + self.weights.c
        p = self.softmax(o)

        assert h.shape[0] == self.hiddenDim
        assert p.shape[0] == self.chars

        return p, h

    def Sample(self, P):
        cp = np.cumsum(P)
        a = np.random.rand()
        ixs = np.where(cp - a > 0)
        ii = ixs[0][0]
        return ii

    def ComputeLoss(self, X, Y_true):
        Y_pred = self.ForwardPass(X)

        # loss over all time steps
        _loss = - Y_true * np.log(Y_pred)
        loss = np.sum(_loss, axis=0).sum()  # mean???

        return np.squeeze(loss)

    def ComputeLossForNumGrad(self, X, Y_true, weigths_try, h0):
        safe = copy.deepcopy(self)

        self.weights = weigths_try
        self.h0 = h0

        res = self.ComputeLoss(X, Y_true)

        # reset everything^^
        self.cache = safe.cache
        self.weights = safe.weights
        self.grads = safe.grads
        self.h0 = safe.h0

        return res

    def SynthesizeText(self, seqLength):
        prediction = []
        x = string2oneHot(self.dummy)
        h = self.h0

        for i in range(seqLength):
            p, _ = self.SingleForwardPass(x, h)
            idx = self.Sample(p)
            prediction.append(index_to_char[idx])
            x = oneHotEncode(idx)

        return prediction

    def ForwardPass(self, X):
        assert X.shape == (self.chars, self.seqLength)

        self.cache = defaultdict(list)

        predictions = []
        hiddens = []
        h = self.h0

        for t in range(self.seqLength):
            x = X[:, t].reshape(-1, 1)

            self.cache["x"].append(x)
            self.cache["h_prev"].append(h)

            p, h = self.SingleForwardPass(x, h)
            hiddens.append(h)
            predictions.append(p)

            self.cache["h"].append(h)
            self.cache["p"].append(p)

        predictions = np.asarray(predictions).squeeze().T
        hiddens = np.asarray(hiddens).squeeze().T

        assert predictions.shape == (self.chars, self.seqLength)
        assert hiddens.shape == (self.hiddenDim, self.seqLength)

        return predictions #, hiddens

    def BackwardPassSingle(self, Y_pred, Y_true, dh_next, h_next, h_prev, xt):
        dy = Y_pred - Y_true

        # gradients wrt y
        self.grads.V += dy @ h_next.T
        self.grads.c += dy
        dh = np.dot(self.weights.V.T, dy) + dh_next

        # gradients wrt hidden
        dtanh = self.tanh(dh, d=True)
        self.grads.b += dtanh
        self.grads.U += dtanh @ xt.T
        self.grads.W += dtanh @ h_prev.T

        dh_next = self.weights.W.T @ dtanh
        return dh_next


    def BackwardPass(self, X, Y_pred, Y_true):
        assert X.shape == Y_pred.shape == Y_true.shape == (self.chars, self.seqLength)

        self.grads = Grads(self.chars, self.hiddenDim) # reset grads

        dY = Y_pred - Y_true    # 83x25

        for i in range(self.seqLength):
            self.grads.V += dY[:, i, None] @ self.cache["h"][i+1].T # gt and ht
        self.grads.c = np.sum(dY, axis=1, keepdims=True)

        assert self.grads.V.shape == self.weights.V.shape # 83x1 @ 1x100 = 83x100
        assert self.grads.c.shape == self.weights.c.shape # 83x1

        dH = [None] * self.seqLength
        dA = [None] * self.seqLength
        for i in reversed(range(self.seqLength-1)):
            # dL / dh = sum of
            ## dL / do
            dH[i] = dY[:, i, None].T @ self.weights.V     # 1x83 @ 83x100 = 1x100
            ## + dL / da_t+1
            dH[i] += self.cache["a"][i+1].T @ self.weights.W    # 1x100 @ 100x100 = 1x100
            assert dH[i].shape == (1, self.hiddenDim)

            # dL / da_t
            dA[i] = dH[i] @ np.diag(1 - np.tanh(self.cache["a"][i][:,0])**2)  # 1x100 @ 100x100 = 1x100
            assert dA[i].shape == (1, self.hiddenDim)

        for i in range(self.seqLength-1):
            self.grads.W += dA[i].T @ self.cache["h"][i].T # gt and ht-1    # 100x1 @ 1x100 = 100x100
            self.grads.U += dA[i].T @ X[:, i, None].T                # 100x1 @ 1x83 = 100x83
            self.grads.b += dA[i].T                                       # 100x1

        assert self.grads.W.shape == self.weights.W.shape
        assert self.grads.U.shape == self.weights.U.shape
        assert self.grads.b.shape == self.weights.b.shape

        # self.ClipGradients()
        return self.grads

    def ComputeGradsNum(self, X, Y, h=1e-4):
        num_grads = Grads(self.chars, self.hiddenDim)

        for f in self.weights.__dict__.keys():
            print('Computing numerical gradient for')
            print(f'Field name: {f}')
            num_grads.__dict__[f] = self.ComputeGradNum(X, Y, f, h)
        return num_grads

    def ComputeGradNum(self, X, Y, f, h):
        grad = np.zeros_like(self.weights.__dict__[f])
        h0 = self.h0

        for i in np.ndindex(self.weights.__dict__[f].shape):
            weights_try = copy.deepcopy(self.weights)
            weights_try.__dict__[f][i] -= h
            l1 = self.ComputeLossForNumGrad(X, Y, weights_try, h0)
            weights_try.__dict__[f][i] += 2 * h
            l2 = self.ComputeLossForNumGrad(X, Y, weights_try, h0)
            grad[i] = (l2 - l1) / (2 * h)
        return grad

    def ClipGradients(self, t = 5):
        for f in self.grads.__dict__.keys():
            self.grads.__dict__[f] = max(min(self.grads.__dict__[f], t), -t)

    def SGDStep(self):
        for f in self.weights.__dict__.keys():
            self.weights.__dict__[f] -= self.eta * self.grads.__dict__[f]

    def Fit(self, X, Y):
        Y_pred = self.ForwardPass(X)
        grads = self.BackwardPass(X, Y_pred, Y)
        global num_grads
        num_grads = self.ComputeGradsNum(X, Y)

        for f in self.grads.__dict__.keys():
            print(f"Asserting {f}")
            # assert np.allclose(grads.__dict__[f], num_grads.__dict__[f], atol=1e-3)

        self.SGDStep()

        loss = self.ComputeLoss(X, Y)
        print(f"Loss = {loss}")


rnn = RNN(K, 5, 25, 1e-1)

X_ = bookContent[0:rnn.seqLength]
Y_ = bookContent[1:rnn.seqLength + 1]

# print(X_)
# print(Y_)

X_ = string2oneHot(X_)
Y_ = string2oneHot(Y_)
print(X_.shape)
print(Y_.shape)

seq = rnn.SynthesizeText(rnn.seqLength)
print("".join(seq))

rnn.Fit(X_, Y_)


(83, 25)
(83, 25)
e Sdl 0d GjCMQ"YeobB/€Plv
Computing numerical gradient for
Field name: b
Computing numerical gradient for
Field name: c
Computing numerical gradient for
Field name: U
Computing numerical gradient for
Field name: W
Computing numerical gradient for
Field name: V
Asserting b
Asserting c
Asserting U
Asserting W
Asserting V
Loss = 105.40734768420859


In [6]:
num_grads.__dict__["V"]

array([[ 1.39757930e-03,  1.37007603e-03,  1.34184319e-03,
         1.03167658e-03,  1.44539122e-03],
       [ 1.39773022e-03,  1.37023363e-03,  1.34195716e-03,
         1.03178749e-03,  1.44552338e-03],
       [-2.07766846e-02, -1.09043265e-02, -1.65785935e-02,
        -8.76061257e-03, -2.44227152e-02],
       [ 1.39765490e-03,  1.37017352e-03,  1.34191325e-03,
         1.03173662e-03,  1.44546185e-03],
       [ 1.39768957e-03,  1.37018176e-03,  1.34193570e-03,
         1.03175175e-03,  1.44550413e-03],
       [ 1.39767764e-03,  1.37018255e-03,  1.34191311e-03,
         1.03174912e-03,  1.44547215e-03],
       [ 1.39760779e-03,  1.37009543e-03,  1.34185427e-03,
         1.03169057e-03,  1.44541957e-03],
       [ 1.39773682e-03,  1.37022305e-03,  1.34197869e-03,
         1.03178721e-03,  1.44554555e-03],
       [ 1.39775914e-03,  1.37027243e-03,  1.34200242e-03,
         1.03180966e-03,  1.44557035e-03],
       [ 1.39763493e-03,  1.37013821e-03,  1.34189705e-03,
         1.03172233e-03

In [220]:
type(num_grads)

In [223]:
print("aa"+type(num_grads))

In [224]:
def CreateData(bookContent, seqLength):
    X = []
    Y = []

    for i in range(len(bookContent) - seqLength):
        X.append(bookContent[i    : i + seqLength])
        Y.append(bookContent[i + 1: i + seqLength + 1])

    X = np.asarray(X)
    X = string2oneHot(X) # would result in list of matrices? do i want that?

    Y = np.asarray(Y)
    Y = string2oneHot(Y)

    return X, Y

# X, Y = CreateData(bookContent, rnn.seqLength)

In [227]:
print("hi")